In [1]:
import os
import re
import pandas as pd
import tiktoken
from pdfminer.high_level import extract_text

In [2]:
#!pwd

In [3]:
# Definir o modelo e obter o codificador apropriado
modelo = 'gpt-4'
try:
    encoding = tiktoken.encoding_for_model(modelo)
except KeyError:
    print(f'Modelo "{modelo}" não encontrado. Usando codificação padrão "cl100k_base".')
    encoding = tiktoken.get_encoding('cl100k_base')

# Lista de caminhos para os arquivos PDF e seus tipos correspondentes
pdf_files = [
    ('/opt/notebooks/xxx/acordao_2249_2024_Plenario.pdf', 'acordao'),
    ('/opt/notebooks/xxxx/instrucao_processo.pdf', 'instrucao_processo')
]

# Lista para armazenar os dados dos PDFs
dados = []

for pdf_path, tipo_de_arquivo in pdf_files:
    # Verificar se o arquivo PDF existe
    if not os.path.isfile(pdf_path):
        print(f'Arquivo não encontrado: {pdf_path}')
        continue

    # Obter o nome base do arquivo
    base_name = os.path.basename(pdf_path)
    # Remover a extensão
    file_name_no_ext = os.path.splitext(base_name)[0]

    print(f'Processando o arquivo: {pdf_path}')
    # Extrair todo o texto do PDF
    texto_completo = extract_text(pdf_path)

    # Limpeza do texto
    texto_limpo = re.sub(r'\n\s*\n+', '\n\n', texto_completo)
    texto_limpo = re.sub(r'[ ]{2,}', ' ', texto_limpo)
    linhas = [linha.strip() for linha in texto_limpo.split('\n') if linha.strip() != '']
    texto_final = '\n'.join(linhas)

    # Calcular o número de tokens
    tokens = encoding.encode(texto_final)
    numero_de_tokens = len(tokens)

    # Adicionar os dados à lista
    dados.append({
        'path': pdf_path,
        'filename': base_name,
        'extracted_text': texto_final,
        'numero_de_tokens': numero_de_tokens,
        'tipo_de_arquivo': tipo_de_arquivo
    })

    print(f'Extração concluída para "{pdf_path}". Número de tokens: {numero_de_tokens}\n')

Processando o arquivo: /opt/notebooks/aceleracao_digital/acordao_2249_2024_Plenario.pdf
Extração concluída para "/opt/notebooks/aceleracao_digital/acordao_2249_2024_Plenario.pdf". Número de tokens: 1807

Processando o arquivo: /opt/notebooks/aceleracao_digital/instrucao_processo.pdf
Extração concluída para "/opt/notebooks/aceleracao_digital/instrucao_processo.pdf". Número de tokens: 17109



In [4]:
# Criar o DataFrame
df = pd.DataFrame(dados)

# Salvar o DataFrame em dois locais diferentes
caminho_arquivo_1 = '/opt/notebooks/aceleracao_digital/IA_recomendacoes_todos_relatorios.xlsx'
caminho_arquivo_2 = '/opt/notebooks/aceleracao_digital/IA_recomendacoes_todos_relatorios2.xlsx'

df.to_excel(caminho_arquivo_1, index=False)
df.to_excel(caminho_arquivo_2, index=False)

print(f"DataFrame salvo em '{caminho_arquivo_1}' e '{caminho_arquivo_2}'")

DataFrame salvo em '/opt/notebooks/aceleracao_digital/IA_recomendacoes_todos_relatorios.xlsx' e '/opt/notebooks/aceleracao_digital/IA_recomendacoes_todos_relatorios2.xlsx'


In [20]:
# Função para simular a chamada à API
def assistenteTCU(prompt):
    # Simular a resposta com base no tipo de documento
    if 'acórdão' in prompt:
        # Retornar o exemplo de JSON para 'acordao'
        return {
            "resultado": [
                {
                    "riscos_e_impactos": "teste...",
                    "responsabilidades_obrigacoes": "teste estejam em conformidade...",
                    "recomendacoes_determinacoes": "testes ao Banco do Brasil...",
                    "prazos_cronogramas": "testea o Banco do Brasil...",
                    "ministros_acordaram": "teste às organizações fiscalizadas...",
                    "resumo_executivo": "O teste d..."
                }
            ]
        }
    elif 'Relatório Técnico' in prompt or 'instrução de processo' in prompt:
        # Retornar o exemplo de JSON para 'instrucao_processo'
        return {
            "resultado": [
                {
                    "citacoes_referencias": "testeerais avaliadas...",
                    "responsabilidades_obrigacoes": "teste asil tem a responsabilidade...",
                    "recomendacoes_acoes": "teste...",
                    "prazos_cronogramas": "teste ...",
                    "riscos_impactos": "Os teste transparência questionada...",
                    "conformidade_regulamentacao": "teste Informação...",
                    "resumo_executivo": "teste PNTP 2023..."
                }
            ]
        }
    else:
        return {}


In [21]:
df

,path,filename,extracted_text,numero_de_tokens,tipo_de_arquivo
0,/opt/notebooks/aceleracao_digital/acordao_2249...,acordao_2249_2024_Plenario.pdf,ACÓRDÃO Nº 2249/2024 - TCU - Plenário\nVISTOS ...,1807,acordao
1,/opt/notebooks/aceleracao_digital/instrucao_pr...,instrucao_processo.pdf,RELATÓRIO DE LEVANTAMENTO\nTRANSPARÊNCIA EM PO...,17109,instrucao_processo


In [22]:
# Listas separadas para armazenar os resultados
resultados_acordao = []
resultados_instrucao_processo = []

for index, row in df.iterrows():
    nome_relatorio = row['filename']
    relatorio = row['extracted_text']
    tipo_de_arquivo = row['tipo_de_arquivo']

    if tipo_de_arquivo.lower() == 'acordao':
        prompt = f"""
        Você receberá um acórdão do TCU relacionado ao Banco do Brasil.

        **Nome do Documento:** {nome_relatorio}

        **Acórdão:**
        {relatorio}

        **Sua tarefa é analisar o acórdão a partir da perspectiva do Banco do Brasil e fornecer as seguintes informações:**

        1. **Identificar os riscos e impactos para o Banco do Brasil:**
           - Riscos legais, financeiros ou operacionais.
           - Penalidades ou sanções potenciais.

        2. **Detalhar as responsabilidades e obrigações atribuídas ao Banco:**
           - Obrigações de cumprimento de prazos.
           - Requisitos para fornecimento de informações ou implementação de medidas.

        3. **Listar as recomendações ou determinações que afetam o Banco:**
           - Como o Banco deve proceder para atender a essas recomendações.

        4. **Destacar quaisquer prazos ou cronogramas importantes:**
           - Datas limites para ações específicas.

        5. **O que os Ministros acordaram?:**
           - O que os Ministros acordaram?

        6. **Fornecer um resumo executivo dos principais pontos relevantes para o Banco do Brasil.**

        **Retorne o resultado no seguinte formato JSON:**

        {{
          "resultado": [
            {{
              "riscos_e_impactos": "Descrição dos riscos e impactos.",
              "responsabilidades_obrigacoes": "Descrição das responsabilidades.",
              "recomendacoes_determinacoes": "Lista de recomendações.",
              "prazos_cronogramas": "Descrição dos prazos.",
              "ministros_acordaram": "Ministros acordaram.",
              "resumo_executivo": "Resumo dos pontos-chave."
            }}
          ]
        }}
        OBS.: Envie o JSON em formato puro sem vestígios de markdown, removendo a marcação abaixo:
        [REMOVER]:
        """.strip()

        # Simular a chamada à API
        resposta = assistenteTCU(prompt)
        # Extrair os dados do JSON
        resultado_data = resposta.get('resultado', [{}])[0]
        # Adicionar informações adicionais
        resultado_data['filename'] = nome_relatorio
        resultado_data['tipo_de_arquivo'] = tipo_de_arquivo
        resultados_acordao.append(resultado_data)

    elif tipo_de_arquivo.lower() == 'instrucao_processo':
        prompt = f"""
        Você receberá uma instrução de processo emitida pelo TCU, relacionada ao Banco do Brasil.

        **Nome do Documento:** {nome_relatorio}

        **Relatório Técnico:**
        {relatorio}

        **Sua tarefa é analisar o relatório técnico a partir da perspectiva do Banco do Brasil e fornecer as seguintes informações:**

        1. **Citações e referências ao Banco do Brasil:**
           - Liste todas as menções diretas ou indiretas ao Banco.
           - Contextualize o significado de cada citação.

        2. **Responsabilidades e obrigações atribuídas ao Banco:**
           - Detalhe quaisquer responsabilidades destacadas.
           - Identifique obrigações legais ou regulatórias.

        3. **Recomendações e ações necessárias:**
           - Liste as ações recomendadas que o Banco deve tomar.
           - Sugira possíveis medidas para cumprir as recomendações.

        4. **Prazos e cronogramas:**
           - Identifique quaisquer prazos estabelecidos no relatório.
           - Destaque datas críticas para o cumprimento das obrigações.

        5. **Riscos e impactos potenciais:**
           - Descreva os riscos identificados que podem afetar o Banco.
           - Analise os impactos financeiros, operacionais ou reputacionais.

        6. **Conformidade e regulamentação:**
           - Avalie questões relacionadas à conformidade com leis e regulamentos.
           - Indique se há necessidade de ajustes em políticas internas.

        7. **Resumo executivo:**
           - Forneça um resumo dos principais pontos relevantes para o Banco do Brasil.

        **Retorne o resultado no seguinte formato JSON:**

        {{
          "resultado": [
            {{
              "citacoes_referencias": "Listagem e contextualização das citações.",
              "responsabilidades_obrigacoes": "Descrição das responsabilidades.",
              "recomendacoes_acoes": "Lista de recomendações e ações necessárias.",
              "prazos_cronogramas": "Descrição dos prazos e datas importantes.",
              "riscos_impactos": "Análise dos riscos e impactos potenciais.",
              "conformidade_regulamentacao": "Avaliação de conformidade e necessidades regulatórias.",
              "resumo_executivo": "Resumo dos pontos-chave."
            }}
          ]
        }}
        OBS.: Envie o JSON em formato puro sem vestígios de markdown, removendo a marcação abaixo:
        [REMOVER]:
        """.strip()

        # Simular a chamada à API
        resposta = assistenteTCU(prompt)
        # Extrair os dados do JSON
        resultado_data = resposta.get('resultado', [{}])[0]
        # Adicionar informações adicionais
        resultado_data['filename'] = nome_relatorio
        resultado_data['tipo_de_arquivo'] = tipo_de_arquivo
        resultados_instrucao_processo.append(resultado_data)

    else:
        pass



In [23]:
resultados_instrucao_processo

[{'riscos_e_impactos': 'teste...',
  'responsabilidades_obrigacoes': 'teste estejam em conformidade...',
  'recomendacoes_determinacoes': 'testes ao Banco do Brasil...',
  'prazos_cronogramas': 'testea o Banco do Brasil...',
  'ministros_acordaram': 'teste às organizações fiscalizadas...',
  'resumo_executivo': 'O teste d...',
  'filename': 'instrucao_processo.pdf',
  'tipo_de_arquivo': 'instrucao_processo'}]

In [24]:
# Criar DataFrames separados para cada tipo de arquivo
df_acordao = pd.DataFrame(resultados_acordao)
df_instrucao_processo = pd.DataFrame(resultados_instrucao_processo)

# Salvar o DataFrame de resultados em dois locais diferentes
caminho_acordao_1 = '/opt/notebooks/aceleracao_digital/IA_analise_resultados.xlsx'
caminho_acordao_2 = '/opt/notebooks/aceleracao_digital/IA_analise_resultados2.xlsx'

df_acordao.to_excel(caminho_acordao_1, index=False)
df_acordao.to_excel(caminho_acordao_2, index=False)

print(f"DataFrame 'acordao' salvo em '{caminho_acordao_1}' e '{caminho_acordao_2}'")


DataFrame 'acordao' salvo em '/opt/notebooks/aceleracao_digital/IA_analise_resultados.xlsx' e '/opt/notebooks/aceleracao_digital/IA_analise_resultados2.xlsx'


In [25]:
df_acordao

,riscos_e_impactos,responsabilidades_obrigacoes,recomendacoes_determinacoes,prazos_cronogramas,ministros_acordaram,resumo_executivo,filename,tipo_de_arquivo
0,teste...,teste estejam em conformidade...,testes ao Banco do Brasil...,testea o Banco do Brasil...,teste às organizações fiscalizadas...,O teste d...,acordao_2249_2024_Plenario.pdf,acordao


In [26]:
# Para 'instrucao_processo'
caminho_instrucao_1 = '/opt/notebooks/aceleracao_digital/IA_analise_instrucao_processo.xlsx'
caminho_instrucao_2 = '/opt/notebooks/aceleracao_digital/IA_analise_instrucao_processo.xlsx'

df_instrucao_processo.to_excel(caminho_instrucao_1, index=False)
df_instrucao_processo.to_excel(caminho_instrucao_2, index=False)

print(f"DataFrame 'instrucao_processo' salvo em '{caminho_instrucao_1}' e '{caminho_instrucao_2}'")

DataFrame 'instrucao_processo' salvo em '/opt/notebooks/aceleracao_digital/IA_analise_instrucao_processo.xlsx' e '/opt/notebooks/aceleracao_digital/IA_analise_instrucao_processo.xlsx'
